# 🗂️ K-Means — Segmentação RFM de Clientes
**Pós-Graduação BI & Analytics · ML Aplicado a Decisões de Negócio**

---

### 📡 Sinal de Negócio
> Taxa de abertura de campanhas caiu de **22% para 9%** em 12 meses.  
> Hipótese do marketing: estamos mandando a mesma mensagem para perfis completamente diferentes.  
> Pergunta: **existem grupos naturais de comportamento de compra?**

### 🏗️ O que vamos construir
Segmentação de clientes baseada em **RFM + variação de compras**.  
A saída é um arquivo com cluster por cliente para alimentar o CRM.

---
### Roteiro
1. Importar e explorar a base
2. Por que normalizar? (demonstração visual)
3. Encontrar o k certo (Elbow + Silhouette)
4. Treinar o K-Means
5. Visualizar (PCA 2D)
6. Interpretar os clusters (centróides)
7. Validar e nomear
8. Cruzar com resultados do Logit
9. Exportar para o CRM

In [ ]:
# ── CÉLULA PRONTA: imports ─────────────────────────────────────────
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, davies_bouldin_score

plt.rcParams.update({'figure.dpi': 120, 'axes.spines.top': False, 'axes.spines.right': False})
CORES_CLUSTER = ['#c84b2f', '#94a3b8', '#2a8c5a', '#7c3dc8']
print('✅ Bibliotecas carregadas')

In [ ]:
# ── CÉLULA PRONTA: carregar dados ──────────────────────────────────
URL = 'https://raw.githubusercontent.com/SEU_USUARIO/ml-bi-analytics-aula/main/data/rfm_clientes.csv'
# df = pd.read_csv('rfm_clientes.csv')  # local
df = pd.read_csv(URL)
print(f'Shape: {df.shape}')
print('\nEstatísticas descritivas:')
df.describe().round(1)

## 2. Por que normalizar?

> **A base foi entregue SEM normalização — propositalmente.**  
> Antes de corrigir isso, vamos ver o que acontece com K-Means sem normalizar.

In [ ]:
# ── CÉLULA PRONTA: demonstração visual do problema ────────────────
FEATURES = ['recencia_dias','freq_compras_ano','valor_total_ano',
            'var_compras_ano_ant','ticket_medio','num_produtos_distintos',
            'nps_score','canal_digital']

X = df[FEATURES].copy()

fig, axes = plt.subplots(1, 2, figsize=(13, 4))

# Escalas sem normalização
scales = X.std().sort_values(ascending=False)
axes[0].barh(scales.index, scales.values, color='#c84b2f')
axes[0].set(title='Desvio Padrão por Feature\n(SEM normalização)',
            xlabel='Desvio Padrão')
for i, v in enumerate(scales.values):
    axes[0].text(v + 10, i, f'{v:,.0f}', va='center', fontsize=8)

# K-Means sem normalizar (demonstrar problema)
km_raw = KMeans(n_clusters=4, random_state=42, n_init=10).fit(X)
axes[1].scatter(X['recencia_dias'], X['valor_total_ano'],
                c=km_raw.labels_, cmap='tab10', s=10, alpha=0.5)
axes[1].set(title='Clusters SEM normalização\n(dominados por valor_total_ano)',
            xlabel='Recência (dias)', ylabel='Valor Total Ano (R$)')

plt.tight_layout()
plt.show()

print('\n💡 valor_total_ano tem desvio padrão ~100× maior que nps_score.')
print('   K-Means usa distância euclidiana → a variável de maior escala domina.')

In [ ]:
# ╔══════════════════════════════════════════════╗
# ║  COMPLETE AQUI — Aplicar StandardScaler     ║
# ╚══════════════════════════════════════════════╝
# Dica: scaler = StandardScaler()
#       X_sc = scaler.fit_transform(X)
#       X_sc = pd.DataFrame(X_sc, columns=FEATURES)

scaler = ___
X_sc   = pd.DataFrame(___, columns=FEATURES)

print('Após normalização — média e desvio padrão:')
print(X_sc.agg(['mean','std']).round(3).to_string())

## 3. Encontrar o k Certo

In [ ]:
# ╔══════════════════════════════════════════════════════════╗
# ║  COMPLETE AQUI — Loop de inércia para o Elbow Method   ║
# ╚══════════════════════════════════════════════════════════╝
# Dica: para k in range(2, 9): km = KMeans(n_clusters=k, random_state=42, n_init=10)
#       inertias.append(km.fit(X_sc).inertia_)

K_range  = range(2, 9)
inertias = []
silhouettes = []

for k in K_range:
    km = KMeans(n_clusters=___, random_state=42, n_init=10)
    km.fit(___)
    inertias.append(___)
    silhouettes.append(silhouette_score(X_sc, km.labels_))

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(K_range, inertias, 'o-', color='#2f6ec8', lw=2, ms=7)
axes[0].set(title='Método do Cotovelo (Elbow)', xlabel='k', ylabel='Inércia')
axes[0].axvline(4, color='#c84b2f', lw=1.5, linestyle='--', label='k=4')
axes[0].legend()

axes[1].plot(K_range, silhouettes, 'o-', color='#2a8c5a', lw=2, ms=7)
axes[1].set(title='Silhouette Score', xlabel='k', ylabel='Score')
axes[1].axvline(K_range[silhouettes.index(max(silhouettes))],
                color='#c84b2f', lw=1.5, linestyle='--',
                label=f'k ótimo={K_range[silhouettes.index(max(silhouettes))]}')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f'Silhouette máximo: {max(silhouettes):.3f} em k={K_range[silhouettes.index(max(silhouettes))]}')
print('\n💡 O cotovelo sugere k=4. Mas k=3 também pode ser interessante.')
print('   A escolha final depende do que o negócio consegue operacionalizar.')

## 4. Treinar o K-Means com k=4

In [ ]:
# ╔═══════════════════════════════════════════════════╗
# ║  COMPLETE AQUI — Treinar KMeans com k=4         ║
# ╚═══════════════════════════════════════════════════╝

K = 4
km = KMeans(n_clusters=___, random_state=42, n_init=15)
km.fit(___)

df['cluster'] = km.labels_

print(f'Silhouette Score (k={K}): {silhouette_score(X_sc, km.labels_):.3f}')
print(f'Davies-Bouldin  (k={K}): {davies_bouldin_score(X_sc, km.labels_):.3f}')
print('\nDistribuição dos clusters:')
print(df['cluster'].value_counts().sort_index())

## 5. Visualizar — PCA 2D

In [ ]:
# ╔═══════════════════════════════════════════╗
# ║  COMPLETE AQUI — PCA 2D + scatter plot  ║
# ╚═══════════════════════════════════════════╝
# Dica: pca = PCA(n_components=2)
#       X_pca = pca.fit_transform(X_sc)

pca   = PCA(n_components=___)
X_pca = pca.fit_transform(___)

fig, ax = plt.subplots(figsize=(9, 6))
for c, cor in zip(range(K), CORES_CLUSTER):
    mask = df['cluster'] == c
    ax.scatter(X_pca[mask, 0], X_pca[mask, 1],
               color=cor, label=f'Cluster {c}', s=12, alpha=0.55)

# Marcar centróides no espaço PCA
centroids_pca = pca.transform(km.cluster_centers_)
ax.scatter(centroids_pca[:, 0], centroids_pca[:, 1],
           c=CORES_CLUSTER[:K], s=200, marker='*',
           edgecolors='black', linewidths=0.8, zorder=5, label='_nolegend_')

ax.set(title=f'K-Means k={K} — Visualização PCA 2D\n'
             f'(variância explicada: {pca.explained_variance_ratio_.sum():.1%})',
       xlabel=f'PC1 ({pca.explained_variance_ratio_[0]:.1%})',
       ylabel=f'PC2 ({pca.explained_variance_ratio_[1]:.1%})')
ax.legend(markerscale=2)
plt.tight_layout()
plt.show()
print('\n💡 PCA projeta 8 dimensões em 2 para visualização.')
print('   Os clusters devem aparecer razoavelmente separados.')

## 6. Interpretar os Clusters — Centróides

In [ ]:
# ╔══════════════════════════════════════════════════════╗
# ║  COMPLETE AQUI — Calcular médias por cluster       ║
# ╚══════════════════════════════════════════════════════╝
# Dica: df.groupby('cluster')[FEATURES].mean().round(2)

centroides = df.groupby('cluster')[___].mean().round(2)
print('Médias por cluster (escala original — antes da normalização):')
centroides

In [ ]:
# ── CÉLULA PRONTA: heatmap de centróides normalizados ─────────────
import matplotlib.colors as mcolors

cent_sc = pd.DataFrame(km.cluster_centers_, columns=FEATURES)

fig, ax = plt.subplots(figsize=(12, 3.5))
im = ax.imshow(cent_sc.values, cmap='RdYlGn', aspect='auto', vmin=-2.5, vmax=2.5)
ax.set(xticks=range(len(FEATURES)), yticks=range(K),
       xticklabels=FEATURES, yticklabels=[f'Cluster {c}' for c in range(K)])
plt.setp(ax.get_xticklabels(), rotation=30, ha='right', fontsize=9)
for i in range(K):
    for j in range(len(FEATURES)):
        val = cent_sc.values[i, j]
        ax.text(j, i, f'{val:.2f}', ha='center', va='center', fontsize=8,
                color='black' if abs(val) < 1.5 else 'white')
plt.colorbar(im, ax=ax, label='z-score', shrink=0.8)
ax.set_title('Centróides Normalizados — Verde=Alto, Vermelho=Baixo', fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# ╔═══════════════════════════════════════════════════╗
# ║  COMPLETE AQUI — Nomear os clusters             ║
# ╚═══════════════════════════════════════════════════╝
# Analise a tabela de centróides acima e proponha um nome para cada cluster
# Exemplos: 'Campeões', 'Em Risco', 'Inativos', 'Novos Prometedores'

NOMES_CLUSTER = {
    0: '___',   # analise: alta/baixa recência? frequência? var_compras?
    1: '___',
    2: '___',
    3: '___',
}

df['cluster_nome'] = df['cluster'].map(NOMES_CLUSTER)

# Verificar concentração de Segmento A por cluster
print('Concentração do Segmento A por cluster:')
print(df.groupby('cluster_nome')['segmento_A'].agg(['mean','sum'])
      .rename(columns={'mean':'%_seg_A','sum':'n_seg_A'})
      .round(3).to_string())
print('\n💡 Qual cluster concentra mais o Segmento A?')
print('   Esse cluster deve ter maior sobreposição com os churners do Logit.')

In [ ]:
# ── CÉLULA PRONTA: boxplots de var_compras por cluster ────────────
fig, axes = plt.subplots(1, 2, figsize=(13, 4))

grupos = [df[df['cluster'] == c]['var_compras_ano_ant'] for c in range(K)]
bp = axes[0].boxplot(grupos, patch_artist=True,
                     medianprops=dict(color='black', lw=2))
for patch, cor in zip(bp['boxes'], CORES_CLUSTER):
    patch.set_facecolor(cor)
    patch.set_alpha(0.7)
axes[0].set(title='Variação de Compras por Cluster',
            xlabel='Cluster', ylabel='var_compras_ano_ant (%)')
axes[0].axhline(0, color='#888', lw=1, linestyle='--')

# Comparação de NPS
grupos_nps = [df[df['cluster'] == c]['nps_score'] for c in range(K)]
bp2 = axes[1].boxplot(grupos_nps, patch_artist=True,
                      medianprops=dict(color='black', lw=2))
for patch, cor in zip(bp2['boxes'], CORES_CLUSTER):
    patch.set_facecolor(cor)
    patch.set_alpha(0.7)
axes[1].set(title='NPS por Cluster', xlabel='Cluster', ylabel='NPS Score')

plt.tight_layout()
plt.show()

## 8. Cruzar com o Logit — Validação Cruzada dos Modelos

In [ ]:
# ── CÉLULA PRONTA: cruzamento K-Means + Logit ─────────────────────
try:
    lista_crm = pd.read_csv('lista_risco_crm.csv')
    cruzado   = df[['id_cliente','cluster','cluster_nome','var_compras_ano_ant']]\
                .merge(lista_crm[['id_cliente','prob_churn','alerta']], on='id_cliente')

    print('Probabilidade média de churn (Logit) por cluster (K-Means):')
    cross = cruzado.groupby('cluster_nome').agg(
        n=('id_cliente','count'),
        prob_churn_media=('prob_churn','mean'),
        pct_alerta=('alerta','mean')
    ).round(3)
    print(cross.to_string())
    print('\n💡 Clusters com alta probabilidade de churn no Logit E')
    print('   com var_compras negativa no K-Means = máxima prioridade.')
except FileNotFoundError:
    print('ℹ️  Execute o notebook de Logit primeiro para o cruzamento.')

## 9. Exportar para o CRM

In [ ]:
# ── CÉLULA PRONTA: exportar segmentação ───────────────────────────
saida_crm = df[['id_cliente','cluster','cluster_nome',
                'var_compras_ano_ant','nps_score','segmento_A',
                'recencia_dias','freq_compras_ano']].copy()
saida_crm.to_csv('segmentacao_clientes.csv', index=False)
print('✅ segmentacao_clientes.csv exportada')
print(f'   {len(saida_crm):,} clientes segmentados em {K} grupos')
print('\nDistribuição final:')
print(saida_crm['cluster_nome'].value_counts().to_string())
print('\n💡 Cada cluster recebe uma régua de comunicação diferente no CRM.')
print('   Peça ao Claude para sugerir uma estratégia para cada grupo.')